In [ ]:
import matplotlib
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch import tensor
from torchvision import datasets, transforms

matplotlib.rc('image', cmap='Greys')

In [ ]:
# configure pandas to print full data sets
pd.options.display.float_format = '{:,.2f}'.format

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 1000)
np.set_printoptions(linewidth=200)
torch.set_printoptions(linewidth=200)


In [ ]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data
train = datasets.MNIST('./mnist_data/', download=True, train=True)
test = datasets.MNIST('./mnist_data/', download=True, train=False)

print(type(train))
print(len(train))
print(len(test))

In [ ]:
# Get a single sample.
sample = train[0]
print(type(sample[0]))
print(type(sample[1]))

print(sample[1])
sample[0]


In [ ]:
image_array = np.array(sample[0])

print(type(image_array))
print(image_array[4:10,4:10])
print(image_array)

In [ ]:
# List comprehension to load all images into an array of tensors.
train_tensors = [tensor(np.array(sample[0])) for sample in train]
test_tensors = [tensor(np.array(sample[0])) for sample in test]

print(type(train_tensors))
print(len(train_tensors))
print(len(test_tensors))

In [ ]:
# Create a Pandas dataframe from the tensor so that we can use the DataFrame's style object to 
# make a nice display.
#from fastbook import *
from fastai.vision.all import *
train_tensor = train_tensors[0]
#print(train_tensor)
image_df = pd.DataFrame(train_tensor)
image_df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')


In [ ]:
image_df.style

In [ ]:
# Turn our tensors into loaders.
train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=False)